In [ ]:
# get the repo data to work with
!rm -rf /kaggle/working/* # remove old stuff
!wget https://github.com/Sail128/NeuRBF/archive/refs/heads/main.zip
!unzip /kaggle/working/main.zip -d /kaggle/working
!cp -r /kaggle/working/NeuRBF-main/* /kaggle/working

# check version 
!pip show cupy
!pip show torch
!pip show torchvision
!nvcc --version

# install additional libraries
!python -m cupyx.tools.install_library --cuda 12.x --library cutensor
!python -m cupyx.tools.install_library --cuda 12.x --library cudnn
!python -m cupyx.tools.install_library --cuda 12.x --library nccl

# install additional packages
!pip install einops matplotlib kornia imageio imageio-ffmpeg opencv-python pysdf PyMCubes trimesh plotly scipy GPUtil scikit-image scikit-learn pykdtree commentjson tqdm configargparse lpips tensorboard torch-ema ninja tensorboardX numpy pandas rich packaging scipy torchmetrics jax pillow plyfile omegaconf

# build and install the gridencoder 
!pip install /kaggle/working/thirdparty/torch_ngp/gridencoder

In [ ]:
# select which studies to run
run_sine_study = False
run_normalization_study = False
run_rbf_type_study = False
run_hidden_dim_study = False
run_num_levels_study = False
run_large_pluto = True

In [ ]:
from main import main
from IPython.display import Image
from copy import copy

if run_large_pluto:
    print("this will take about 20 minutes")
    !python /kaggle/working/main.py --config configs/img.py --path /kaggle/working/dataset/pluto/pluto.png --alias pluto
    Image(filename="/kaggle/working/log/img/results/pluto-v0.png")

In [ ]:

class Object(object):
    pass

config = Object()
config.config_fp = "configs/img.py"
config.path = "/kaggle/input/pluto-img/Vincent.jpg"
config.alias = "vincent1"
config.ds_device = 'auto'
config.log2_hashmap_size_ref = 24

config.task = 'image'
config.workspace = f'log/img'
config.test = False
config.seed = None
config.cfactor = 128
config.train_shuffle_mode = 2
config.train_num_samples = 2**18
config.record_training = False

config.pe_lc0_freq = [1e1, 1e2]
config.pe_lc0_rbf_freq = [2**-3, 2**12]
config.num_levels = 2
config.level_dim = 2

# Model
config.arch = 'rbf'
config.rbf_type = 'ivq_a'
config.rbf_lc0_normalize = True
config.n_kernel = 'auto'
config.point_nn_kernel = 4
config.ks_alpha = 1
config.n_hidden_fl = 32

config.num_levels_ref = 16
config.level_dim_ref = 2
config.base_resolution_ref = 16

config.log2_hashmap_size = config.log2_hashmap_size_ref
config.base_resolution = 16
config.levels_omit = []

config.pe_freqs = []
config.pe_hg0_freq = []
config.pe_lc0_rbf_keep = 0

config.num_layers = 3
config.hidden_dim = 64
config.lc_act = 'none'
config.act = 'relu'
config.lc_init = [1e-4]
config.lcb_init = [1e-4]
config.w_init = [None, None, None]
config.b_init = [None, None, None]
config.a_init = [9, 30, 30, 30]

config.rbf_suffixes = ['0']

config.fix_params = ['kc0', 'ks0']
config.fix_params += ['a0', 'a1', 'a2', 'a3']

config.sine_before_mlp = True
config.sine_after_rbf=True

# Special
config.ema_decay = None
config.fp16 = False

# Training
config.max_steps = 35000
config.val_freq = 1.0
config.val_first = False
config.log_train = True
config.log_img = False
config.log_kernel_img = False
config.log_sdf_slice = False
config.save_pred = True
config.save_pred_pt = False
config.save_ckpt = False
config.train_metric_list = ['psnr', 'mae']
config.val_metric_list = ['psnr', 'mae', 'mse', 'ssim_ski', 'ms-ssim', 'lpips_alex', 'lpips_vgg']

# Optimizers
eps = 1e-15
lr = 1e-2/2
config.optims = {}
config.optims["dec"] = {'type': 'Adam', 'lr': 1e-2, 'betas': (0.9, 0.99), 'eps': eps, 'wd': 0}
config.optims["hg0"] = {'type': 'Adam', 'lr': lr, 'betas': (0.9, 0.99), 'eps': eps, 'wd': 0}
config.optims["lc0"] = {'type': 'Adam', 'lr': lr, 'betas': (0.9, 0.99), 'eps': eps, 'wd': 0}
config.optims["lcb0"] = {'type': 'Adam', 'lr': lr, 'betas': (0.9, 0.99), 'eps': eps, 'wd': 0}

# lr schedulers
T_max = config.max_steps
lr_gamma = 100
config.lr_schs = {}
config.lr_schs["dec"] = {'type': 'cosine', 'T_max': T_max, 'gamma': lr_gamma}
config.lr_schs["hg0"] = {'type': 'cosine', 'T_max': T_max, 'gamma': lr_gamma}
config.lr_schs["lc0"] = {'type': 'cosine', 'T_max': T_max, 'gamma': lr_gamma}
config.lr_schs["lcb0"] = {'type': 'cosine', 'T_max': T_max, 'gamma': lr_gamma}

# RBF params init
config.kc_init_config = {}
config.kc_init_config['0'] = {'type': 'v3', 'points_sampling': 1, 'reg_sampling': 0, 'weight_exp': 1, 'weight_thres': 0, 'n_iter': 10, 'weight_src': 'gt_grad'}

config.kw_init_config = {}
config.kw_init_config['0'] = {'type': 'v3', 'points_sampling': 1, 'alpha': 0.5, 'weight_exp': 1, 'weight_thres': 0, 'weight_src': 'gt_grad'}

config.save_fn = f'{config.alias}-{config.arch}'

config.exp_name = f'{config.alias}-{config.arch}'

In [ ]:
if run_rbf_type_study:

    images = [10,12,20,22,36,44]

    # rbf_types = ['nlin_s', 'nlin_d', 'nlin_f', 'ivq_s', 'ivq_d', 'ivq_a', 'ivmq_a', 'ivd_a', 'ivc_a', 'gauss_a', 'sgauss2_a', 'nsgauss2_a', 'markov_a', 'expsin_a', 'qd_a', 'mqd_a', 'phs1_a', 'phs1_s', 'phs3_a', 'vep_a', 'cos_s']

    rbf_types = ["nlin_f", "ivq_a", "ivmq_a", "gauss_a", "mqd_a","expsin_a"]

    for img in images:
        print(f"starting image {img}")   
        print("starting rbf_types")
        for rbf_type in rbf_types:
            cfg = copy(config)
            img_name = f"img_{img}"
            cfg.max_steps = 3500
            cfg.path = f"/kaggle/working/dataset/large_images/{img_name}.png"
            cfg.alias= f"rbf-{img_name}-rbftype_{rbf_type}-max_step_{cfg.max_steps}"
            cfg.save_fn = f'{cfg.alias}-{cfg.arch}'
            cfg.exp_name = f'{cfg.alias}-{cfg.arch}'
            main(cfg)

In [ ]:
if run_sine_study:
    images = range(1, 51)

    option_combinations = {
        "sine_rbf_only": {"sine_mlp": False, "sine_rbf": True},
        "sine_mlp_only": {"sine_mlp": True, "sine_rbf": False},
        "no_sine": {"sine_mlp": False, "sine_rbf": False},
    }

    for option_name, option in option_combinations.items():
        cfg = copy(config)
        cfg.sine_before_mlp = option["sine_mlp"]
        cfg.sine_after_rbf= option["sine_rbf"]
        cfg.max_steps = 3500
        
        for img in images:
            img_name = f"img_{img}"
            cfg.path = f"/kaggle/working/dataset/large_images/{img_name}.png"
            cfg.alias= f"{img_name}-max_steps_{cfg.max_steps}-{option_name}"
            cfg.save_fn = f'{cfg.alias}-{cfg.arch}'
            cfg.exp_name = f'{cfg.alias}-{cfg.arch}'
            main(cfg)

In [ ]:
if run_normalization_study:
    images = range(1,51)

    # max_steps = range(500,10000,500)

    for img in images:
        cfg = copy(config)
        
        cfg.max_steps = 3500
        # cfg.sine_after_rbf = False
        # cfg.rbf_lc0_normalize = False
        
        img_name = f"img_{img}"
        cfg.path = f"/kaggle/working/dataset/large_images/{img_name}.png"
        
        # cfg.alias= f"{img_name}-max_steps_{cfg.max_steps}-norm_False"
        # cfg.alias= f"{img_name}-max_steps_{cfg.max_steps}-bias_corrected"
        # cfg.alias= f"{img_name}-max_steps_{cfg.max_steps}-sine_mlp_only"
        cfg.alias= f"{img_name}-max_steps_{cfg.max_steps}-bias_corrected"
        
        cfg.save_fn = f'{cfg.alias}-{cfg.arch}'
        cfg.exp_name = f'{cfg.alias}-{cfg.arch}'

        main(cfg)

In [ ]:
if run_hidden_dim_study:
    images = range(1,51)

    #max_steps = range(500,10000,500)

    for img in images:
        for n in [16, 32]:
            cfg = copy(config)
            img_name = f"img_{img}"
            cfg.path = f"/kaggle/working/dataset/large_images/{img_name}.png"
            cfg.max_steps = 3500
            cfg.alias= f"{img_name}-max_steps_{cfg.max_steps}-hidden_dim_{n}"
            cfg.save_fn = f'{cfg.alias}-{cfg.arch}'
            cfg.exp_name = f'{cfg.alias}-{cfg.arch}'    
            cfg.hidden_dim = n

            main(cfg)

In [ ]:
if run_num_levels_study:
    from copy import copy

    images = range(1,51)

    #max_steps = range(500,10000,500)

    for img in images:
        n = 0
        cfg = copy(config)
        img_name = f"img_{img}"
        cfg.path = f"/kaggle/working/dataset/large_images/{img_name}.png"
        cfg.max_steps = 3500
        cfg.alias= f"{img_name}-max_steps_{cfg.max_steps}-num_levels_{n}"
        cfg.save_fn = f'{cfg.alias}-{cfg.arch}'
        cfg.exp_name = f'{cfg.alias}-{cfg.arch}'    
        cfg.num_levels = n
        main(cfg)


In [ ]:
!zip -r image_results_all.zip log